# Recommendation System

In [1]:
#Libraries
import pandas as pd
import numpy as np

In [2]:
#Load the dataset
data=pd.read_csv("facebook_combined.txt",sep=" ", header=None)

#Add column names
data.columns = ["node1", "node2"]

In [3]:
#Transform the graph to undirected
data2=pd.concat([data.node2,data.node1], axis=1)
data=data.append(data2)

In [110]:
#Create a sample graph dataset
sample = pd.DataFrame([[5, 2], 
                       [9, 3],
                       [9, 11],
                       [3, 6],
                       [4, 6],
                       [5, 7],
                       [1, 11],
                       [6, 2],
                       [7, 9],
                       [8, 9],
                       [5, 11],
                       [6, 7],
                       [6, 11],
                       [7, 6]],
                      columns=["node1", "node2"])

#### Recommending friends using Common neighbors (friend-of-friend (FoF) method)

In [81]:
##### Create the function for Common neighbors

def friendOfFriend(users, dataset):
    #Initialize
    l=list()
    friendships={}

    #Create friendships dict
    for node in users:
        #Create a list with the friends of node
        ls=dataset[dataset.node1 == node]['node2'].tolist()

        #Create a dictionary with key the node and value the list
        friendships[node]=ls

    # Initialize a dictionary with the intersections
    inter={}

    #Intersection between users
    for i in friendships:
        intersection=[0]
        for j in friendships:
            if (i != j) and (i not in friendships[j]) :
                intersection.append(len(set(friendships.get(i)).intersection(set(friendships.get(j)))))
        inter[i]=max(intersection)
        
    #Create a sorted list, in ties we take the smallest ID
    lis=sorted(inter, key=inter.get, reverse=True)

    #Final Result
    print(lis[0:10])
    

In [85]:
##### Test the code for the Sample
users=[1,2,3,4,5,6,7,8,9,10,11]
friendOfFriend(users, sample)

[5, 6, 1, 3, 4, 7, 8, 9, 2, 10]


In [83]:
##### Test the code for the Original Dataset
users=range(0,4038)
friendOfFriend(users, data)

[1912, 107, 1917, 1943, 1938, 1985, 1946, 1983, 1962, 1993]


###  Recommending friends using Jaccard coefficient

In [99]:
##### Create the function for Common neighbors using Jaccard coefficient

def JaccardCoefficient(users, dataset):
    #Initialize
    l=list()
    friendships={}

    #Create friendships dict
    for node in users:
        #Create a list with the friends of node
        ls=dataset[dataset.node1 == node]['node2'].tolist()

        #Create a dictionary with key the node and value the list
        friendships[node]=ls

    # Initialize a dictionary with the intersections
    inter={}

   #Intersection between users
    for i in friendships:
        intersection=[0]
        for j in friendships:
            
            if (i != j) and (i not in friendships[j]) :
                # Create union
                union=len(set(friendships.get(i)).union(set(friendships.get(j))))
                # Check for No zero denominator
                if (union != 0) :
                    intersection.append(len(set(friendships.get(i)).intersection(set(friendships.get(j))))/union)

        inter[i]=max(intersection)

        
    #Create a sorted list, in ties we take the smallest ID
    lis=sorted(inter, key=inter.get, reverse=True)

    #Final Result
    print(lis[0:10])
    

In [100]:
##### Test the code for the Sample
users=[1,2,3,4,5,6,7,8,9,10,11]
JaccardCoefficient(users, sample)

[3, 4, 5, 6, 1, 7, 8, 9, 2, 10]


In [101]:
##### Test the code for the Original Dataset
users=range(0,4038)
JaccardCoefficient(users, data)

[90, 145, 160, 162, 193, 201, 202, 206, 207, 217]


### Recommending friends using Adamic and Adar function

In [148]:
def AdamicAdarFunction(users, dataset):
    #Initialize
    l=list()
    friendships={}

    #Create friendships dict
    for node in users:
        #Create a list with the friends of node
        ls=dataset[dataset.node1 == node]['node2'].tolist()

        #Create a dictionary with key the node and value the list
        friendships[node]=ls

    # Initialize a dictionary with the intersections
    inter={}

    #Intersection between users
    for i in friendships:
        intersection=[0]
        score=[0]
        for j in friendships:
            if (i != j) and (i not in friendships[j]) :
                intersection = set(friendships.get(i)).intersection(set(friendships.get(j)))

                # Adamic and Adar score calculation
                sum = 0
                for k in intersection :
                    if ( k in friendships.keys()) and (friendships[k] != []):
                        sum = sum+1/np.log(len(friendships[k]))
                score.append(sum)

        inter[i]=max(score)
    #Create a sorted list, in ties we take the smallest ID
    lis=sorted(inter, key=inter.get, reverse=True)

    #Final Result
    print(lis[0:10])                  

In [149]:
##### Test the code for the Sample
users=[1,2,3,4,5,6,7,8,9,10,11]
AdamicAdarFunction(users, sample)

[5, 6, 7, 8, 3, 4, 1, 2, 9, 10]


In [150]:
##### Test the code for the Original Dataset
users=range(0,4038)
AdamicAdarFunction(users, data)

[107, 1912, 1577, 1985, 1917, 1983, 1943, 1938, 1993, 1962]
